##CREACIÓN DE UN DATAFRAME DESDE CERO

In [0]:
# creamos una lista desde 0, con valores string, int y además valores nulos
datos = [(None,'Smith   ','36636','M',3500),
         ('Michael','   Rose','40288','M',4750),
         ('Robert','Williams','42114','M',None),
         ('Maria','    Jones    ','39192','F',4000)
        ]

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
# pyspark necesita un esquema para organizarse. lo definimos:
esquema = StructType([
    StructField('firstname', StringType(), True),
    StructField('lastname', StringType(), False),
    StructField('id', StringType(), False),
    StructField('gender', StringType(), True),
    StructField('salary', IntegerType(), True)
])

Out[7]: StructType([StructField('firstname', StringType(), True), StructField('lastname', StringType(), False), StructField('id', StringType(), False), StructField('gender', StringType(), True), StructField('salary', IntegerType(), True)])

In [0]:
# creamos el dataframe con pyspark con los datos y su esquema relacionado
df = spark.createDataFrame(datos, esquema)
# mostramos el esquema del dataframe
df.printSchema()

root
 |-- firstname: string (nullable = true)
 |-- lastname: string (nullable = false)
 |-- id: string (nullable = false)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)



In [0]:
# mostramos el dataframe con todo ya definido
df.show()

+---------+-------------+-----+------+------+
|firstname|     lastname|   id|gender|salary|
+---------+-------------+-----+------+------+
|     null|     Smith   |36636|     M|  3500|
|  Michael|         Rose|40288|     M|  4750|
|   Robert|     Williams|42114|     M|  null|
|    Maria|    Jones    |39192|     F|  4000|
+---------+-------------+-----+------+------+



##CREAR DATAFRAME DESDE ARCHIVO (I)

In [0]:
# asignamos la variable la ruta del archivo que queremos importar para crear csv. esto viene dado desde databricks
file = 'dbfs:/FileStore/shared_uploads/jesuspt98@gmail.com/sales.csv'

In [0]:
# creamos el dataframe desde el archivo que hemos subido
sales_df = spark.read.format("csv").option("header", True).option("inferSchema", True).load(file) # con "inferSchema" el esquema lo deduce pyspark del .csv
# esto comentado viene desde el propio databricks que te da la opción de crear el dataframe directamente desde el archivo justo al subirlo
# df1 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/jesuspt98@gmail.com/sales-1.csv")
# mostramos el esquema del dataframe que hemos subido, que ya estaría asignado dentro del propio archivo
sales_df.printSchema()

root
 |-- Order_ID: integer (nullable = true)
 |-- Order_Date: date (nullable = true)
 |-- Item_Type: string (nullable = true)
 |-- Units_Sold: integer (nullable = true)
 |-- Unit_Price: double (nullable = true)
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Ship_Date: date (nullable = true)



In [0]:
sales_df.show(truncate=False)

+---------+----------+---------------+----------+----------+---------------------------------+----------------------+----------+
|Order_ID |Order_Date|Item_Type      |Units_Sold|Unit_Price|Region                           |Country               |Ship_Date |
+---------+----------+---------------+----------+----------+---------------------------------+----------------------+----------+
|535113847|2014-10-08|Snacks         |934       |152.58    |Middle East and North Africa     |Azerbaijan            |2014-10-23|
|874708545|2015-02-22|Cosmetics      |4551      |437.2     |Central America and the Caribbean|Panama                |2015-02-27|
|854349935|2015-12-09|Fruits         |9986      |9.33      |Sub-Saharan Africa               |Sao Tome and Principe |2016-01-18|
|892836844|2014-09-17|Personal Care  |9118      |81.73     |Sub-Saharan Africa               |Sao Tome and Principe |2014-10-12|
|129280602|2010-02-04|Household      |5858      |668.27    |Central America and the Caribbean|Bel

##CREAR DATAFRAME DESDE ARCHIVO (II)

In [0]:
from pyspark.sql.types import IntegerType, StringType, FloatType, ArrayType, DateType, BooleanType
# ahora lo que vamos a hacer es lo mismo pero con un tipo de archivo distinto, esta vez en .json. establecemos el esquema de la estructura, ya que al ser .json no podemos extraerla del archivo directamente
persons_schema = StructType([
    StructField('id', IntegerType(), True),
    StructField('first_name', StringType(), True),
    StructField('last_name', StringType(), True),
    StructField('fav_movies', ArrayType(StringType()), True),
    StructField('salary', FloatType(), True),
    StructField('image_url', StringType(), True),
    StructField('date_of_birth', DateType(), True),
    StructField('active', BooleanType(), True)
])


In [0]:
# asignamos a la variable del archivo la ruta del archivo que hemos subido
file = 'dbfs:/FileStore/shared_uploads/jesuspt98@gmail.com/persons.json'

In [0]:
# creamos el dataframe en pyspark:
persons_df = spark.read.format('json').option('multiline', True).schema(persons_schema).load(file)
# esta siguiente línea de código nos la da databricks una vez subimos el archivo, pero nos faltan cosas como en la línea anterior: 
# df1 = spark.read.format("json").load("dbfs:/FileStore/shared_uploads/jesuspt98@gmail.com/persons.json")
persons_df.show()

+---+----------+-----------+--------------------+-------+--------------------+-------------+------+
| id|first_name|  last_name|          fav_movies| salary|           image_url|date_of_birth|active|
+---+----------+-----------+--------------------+-------+--------------------+-------------+------+
|  1|     Drucy|      Poppy|  [I giorni contati]|1463.36|http://dummyimage...|   1991-02-16|  true|
|  2|   Emelyne|      Blaza|[Musketeer, The, ...|3006.04|http://dummyimage...|   1991-11-02| false|
|  3|       Max|     Rettie|[The Forgotten Sp...|1422.88|http://dummyimage...|   1990-03-03| false|
|  4|    Ilario|       Kean|[Up Close and Per...|3561.36|http://dummyimage...|   1987-06-09|  true|
|  5|     Toddy|     Drexel|[Walk in the Clou...|4934.87|http://dummyimage...|   1992-10-28|  true|
|  6|    Oswald|   Petrolli|[Wing and the Thi...|1153.23|http://dummyimage...|   1986-09-02| false|
|  7|    Adrian|     Clarey|[Walking Tall, Pa...|1044.73|http://dummyimage...|   1971-08-24| false|


In [0]:
# con el siguiente comando podemos observar el dataframe como en pandas o en excel, de una manera más sencilla de visualizar: 
display(persons_df)

id,first_name,last_name,fav_movies,salary,image_url,date_of_birth,active
1,Drucy,Poppy,List(I giorni contati),1463.36,http://dummyimage.com/126x166.png/cc0000/ffffff,1991-02-16,true
2,Emelyne,Blaza,"List(Musketeer, The, Topralli)",3006.04,http://dummyimage.com/158x106.bmp/cc0000/ffffff,1991-11-02,false
3,Max,Rettie,"List(The Forgotten Space, Make It Happen)",1422.88,http://dummyimage.com/237x140.jpg/ff4444/ffffff,1990-03-03,false
4,Ilario,Kean,List(Up Close and Personal),3561.36,http://dummyimage.com/207x121.jpg/cc0000/ffffff,1987-06-09,true
5,Toddy,Drexel,"List(Walk in the Clouds, A)",4934.87,http://dummyimage.com/116x202.png/cc0000/ffffff,1992-10-28,true
6,Oswald,Petrolli,"List(Wing and the Thigh, The (L'aile ou la cuisse))",1153.23,http://dummyimage.com/137x172.jpg/5fa2dd/ffffff,1986-09-02,false
7,Adrian,Clarey,"List(Walking Tall, Paradise, Hawaiian Style)",1044.73,http://dummyimage.com/244x218.bmp/cc0000/ffffff,1971-08-24,false
8,Dominica,Goodnow,List(Hearts Divided),1147.76,http://dummyimage.com/112x203.jpg/dddddd/000000,1973-08-27,false
9,Emory,Slocomb,"List(Snake and Crane Arts of Shaolin (She hao ba bu), Mala Noche)",1082.11,http://dummyimage.com/138x226.jpg/cc0000/ffffff,1974-06-08,true
10,Jeremias,Bode,"List(Farewell to Arms, A)",3472.63,http://dummyimage.com/243x108.bmp/dddddd/000000,1997-08-02,true


###CONSULTA DE DATOS DF

In [0]:
# los mismos comandos de SQL sirven para pyspark - el select nos selecciona las columnas que deseamos:
sales_df.select('Order_ID','Item_Type','Units_Sold','Unit_Price','Country').show(10,truncate=False) # fijarse en la manera de llamar columnas con () y no en []
# en .show() aclaramos el número de filas que queremos mostrar y el truncate = False nos trunca hacia la izquierda las columnas, si fuese True, hacia la derecha

+---------+-------------+----------+----------+---------------------+
|Order_ID |Item_Type    |Units_Sold|Unit_Price|Country              |
+---------+-------------+----------+----------+---------------------+
|535113847|Snacks       |934       |152.58    |Azerbaijan           |
|874708545|Cosmetics    |4551      |437.2     |Panama               |
|854349935|Fruits       |9986      |9.33      |Sao Tome and Principe|
|892836844|Personal Care|9118      |81.73     |Sao Tome and Principe|
|129280602|Household    |5858      |668.27    |Belize               |
|473105037|Clothes      |1149      |109.28    |Denmark              |
|754046475|Cosmetics    |7964      |437.2     |Germany              |
|772153747|Fruits       |6307      |9.33      |Turkey               |
|847788178|Snacks       |8217      |152.58    |United Kingdom       |
|471623599|Cosmetics    |2758      |437.2     |Kazakhstan           |
+---------+-------------+----------+----------+---------------------+
only showing top 10 

In [0]:
# podemos seleccionar las columnas también con df.COLUMNA como en pandas, igual
sales_df.select(sales_df.Order_ID).show(10,truncate=False)

+---------+
|Order_ID |
+---------+
|535113847|
|874708545|
|854349935|
|892836844|
|129280602|
|473105037|
|754046475|
|772153747|
|847788178|
|471623599|
+---------+
only showing top 10 rows



In [0]:
# también podemos usar distintas funciones de sql que podemos importar desde la librería como col y expr, que son columnas y expresiones 
from pyspark.sql.functions import col, expr
# las usamos así, donde en col seleccionamos la columna y en expr generamos una columna con una expresión, en este caso de unidades vendidas * precio de unidades para dar un precio total
sales_df.select(col('Order_ID'), col('Item_Type'), expr("Units_Sold * Unit_Price as TOTAL_PRICE")).show(10)

+---------+-------------+------------------+
| Order_ID|    Item_Type|       TOTAL_PRICE|
+---------+-------------+------------------+
|535113847|       Snacks|         142509.72|
|874708545|    Cosmetics|         1989697.2|
|854349935|       Fruits|          93169.38|
|892836844|Personal Care|         745214.14|
|129280602|    Household|3914725.6599999997|
|473105037|      Clothes|         125562.72|
|754046475|    Cosmetics|         3481860.8|
|772153747|       Fruits|          58844.31|
|847788178|       Snacks|        1253749.86|
|471623599|    Cosmetics|1205797.5999999999|
+---------+-------------+------------------+
only showing top 10 rows



In [0]:
# podemos obviamente también aplicarles filtros como el where de sql y lo ejecutamos con filter ANTES de seleccionar las columnas, así: 
sales_df.filter((col('Region')=='Europe') & (col('Country')=='Spain')) \
.select(col('Order_ID'), col('Country'), col('Item_Type'),expr("Units_Sold * Unit_Price as TOTAL_PRICE")).show(5)
# & como AND, | como OR, ~ como NOT

+---------+-------+-------------+------------------+
| Order_ID|Country|    Item_Type|       TOTAL_PRICE|
+---------+-------+-------------+------------------+
|860891091|  Spain|Personal Care|462591.80000000005|
|413236844|  Spain|    Household|         3221061.4|
|621470248|  Spain|      Clothes|         624207.36|
|337587821|  Spain|   Vegetables|        1323837.58|
|420354354|  Spain|       Snacks|         160056.42|
+---------+-------+-------------+------------------+
only showing top 5 rows



In [0]:
# así como podemos utilizar el .filter primero, también podemos usar el .where como en SQL con la diferencia que el where solamente admite un filtrado, hay que aplicar otro .where después con la siguiente condición
sales_df.where(col('Region')=='Europe').where(col('Country')=='Spain') \
.select(col('Order_ID'),col('Country'),col('Item_Type'), expr("Units_Sold * Unit_Price as TOTAL_PRICE")).show(5)


+---------+-------+-------------+------------------+
| Order_ID|Country|    Item_Type|       TOTAL_PRICE|
+---------+-------+-------------+------------------+
|860891091|  Spain|Personal Care|462591.80000000005|
|413236844|  Spain|    Household|         3221061.4|
|621470248|  Spain|      Clothes|         624207.36|
|337587821|  Spain|   Vegetables|        1323837.58|
|420354354|  Spain|       Snacks|         160056.42|
+---------+-------+-------------+------------------+
only showing top 5 rows



In [0]:
# podemos también ordenar las columnas por valores ascendentes o descendentes con .orderBy DESPUÉS de seleccionar las columnas
sales_df.select(col('Order_ID'),col('Country'),col('Item_Type'),col('Units_Sold')) \
.orderBy(col('Units_Sold').desc(),col('Country').asc()).show(20,truncate=False) # esto ordena por unidades vendidas, y con los mismos valores de unidades vendidas, ordena por país ascendente
# para tratar con valores nulos existen las opciones ‘asc_nulls_first()’, ‘desc_nulls_first()’, ‘asc_nulls_last()’, ‘desc_nulls_last()’

+---------+------------------------------+---------------+----------+
|Order_ID |Country                       |Item_Type      |Units_Sold|
+---------+------------------------------+---------------+----------+
|257909476|Cape Verde                    |Household      |10000     |
|261322534|Comoros                       |Cosmetics      |10000     |
|122941577|Federated States of Micronesia|Vegetables     |10000     |
|225874030|Ghana                         |Cereal         |10000     |
|143555104|Iceland                       |Personal Care  |10000     |
|573532950|Iran                          |Snacks         |10000     |
|885743367|Panama                        |Meat           |10000     |
|230469834|Qatar                         |Meat           |10000     |
|240709006|Andorra                       |Beverages      |9999      |
|895982539|Cameroon                      |Beverages      |9999      |
|572350203|Cote d'Ivoire                 |Snacks         |9999      |
|264735591|Cote d'Iv

In [0]:
# podemos usar también el distinct de SQL para mostrar los valores únicos de las columnas. en este caso contamos los valores únicos de la columna Region:
print(sales_df.select('Region').distinct().count())

7


In [0]:
# ahora mostramos las regiones distintas: 
sales_df.select('Region').distinct().show(truncate=False)

+---------------------------------+
|Region                           |
+---------------------------------+
|Middle East and North Africa     |
|Australia and Oceania            |
|Europe                           |
|Sub-Saharan Africa               |
|Central America and the Caribbean|
|North America                    |
|Asia                             |
+---------------------------------+



In [0]:
# también podemos limitar/RECORTAR a los números que queremos mostrar con .limit como sql. aquí contamos las filas que tenemos para observar que se limita perfetamente
sales_df.select(col('Order_ID'),col('Country'),col('Item_Type'),col('Units_Sold')) \
.orderBy(col('Units_Sold').desc(),col('Country').asc()).limit(20).count()

Out[66]: 20

In [0]:
# donde aquí tenemos el dataframe de donde hemos limitado/recortado
sales_df.select(col('Order_ID'),col('Country'),col('Item_Type'),col('Units_Sold')) \
.orderBy(col('Units_Sold').desc(),col('Country').asc()).limit(20).show(truncate=False)

+---------+------------------------------+---------------+----------+
|Order_ID |Country                       |Item_Type      |Units_Sold|
+---------+------------------------------+---------------+----------+
|257909476|Cape Verde                    |Household      |10000     |
|261322534|Comoros                       |Cosmetics      |10000     |
|122941577|Federated States of Micronesia|Vegetables     |10000     |
|225874030|Ghana                         |Cereal         |10000     |
|143555104|Iceland                       |Personal Care  |10000     |
|573532950|Iran                          |Snacks         |10000     |
|885743367|Panama                        |Meat           |10000     |
|230469834|Qatar                         |Meat           |10000     |
|240709006|Andorra                       |Beverages      |9999      |
|895982539|Cameroon                      |Beverages      |9999      |
|572350203|Cote d'Ivoire                 |Snacks         |9999      |
|264735591|Cote d'Iv

##EJERCICIO
###     Devolver los campos producto, unidades vendidas, fechas de pedido y envío, de las ventas de la Zona Logística de Asia, ordenadas por país. Sólo nos interesan los 10 primeros.

In [0]:
sales_df.where(col('Region')=='Asia') \
.select(col('Item_Type'),col('Units_Sold'),col('Order_Date'),col('Ship_Date')) \
.orderBy(col('Country').desc()).show(10,truncate=False)

+---------------+----------+----------+----------+
|Item_Type      |Units_Sold|Order_Date|Ship_Date |
+---------------+----------+----------+----------+
|Baby Food      |1702      |2014-04-14|2014-04-22|
|Clothes        |9353      |2013-05-09|2013-06-09|
|Personal Care  |1898      |2013-04-27|2013-06-13|
|Cosmetics      |8397      |2010-07-16|2010-08-11|
|Fruits         |7747      |2017-01-11|2017-02-25|
|Vegetables     |4335      |2010-11-08|2010-11-23|
|Cosmetics      |7990      |2015-11-22|2015-12-16|
|Snacks         |7953      |2015-11-17|2015-12-28|
|Snacks         |5911      |2011-09-11|2011-10-19|
|Office Supplies|9041      |2013-07-23|2013-08-08|
+---------------+----------+----------+----------+
only showing top 10 rows



##MODIFICAR DATOS DATAFRAME

In [0]:
# ahora podemos añadir nuevas columnas con la librería de funciones de sql, con la función lit
from pyspark.sql.functions import lit
# creamos una columna llamada Sent con la función lit como False
# VALOR DETERMINADO
sales_df.withColumn("Sent", lit(False)).show(5)
# creamos una columna nueva en el dataframe con una expresión de unidades vendidas * precio de unidades con la siguiente sintaxis
# CAMPO CALCULADO
sales_df.withColumn("Total_Price", expr("round((Units_Sold *  Unit_Price),2)")).show(5)

+---------+----------+-------------+----------+----------+--------------------+--------------------+----------+-----+
| Order_ID|Order_Date|    Item_Type|Units_Sold|Unit_Price|              Region|             Country| Ship_Date| Sent|
+---------+----------+-------------+----------+----------+--------------------+--------------------+----------+-----+
|535113847|2014-10-08|       Snacks|       934|    152.58|Middle East and N...|          Azerbaijan|2014-10-23|false|
|874708545|2015-02-22|    Cosmetics|      4551|     437.2|Central America a...|              Panama|2015-02-27|false|
|854349935|2015-12-09|       Fruits|      9986|      9.33|  Sub-Saharan Africa|Sao Tome and Prin...|2016-01-18|false|
|892836844|2014-09-17|Personal Care|      9118|     81.73|  Sub-Saharan Africa|Sao Tome and Prin...|2014-10-12|false|
|129280602|2010-02-04|    Household|      5858|    668.27|Central America a...|              Belize|2010-03-05|false|
+---------+----------+-------------+----------+---------

In [0]:
print(sales_df.printSchema())
# podemos cambiarle el nombre a una columa con .withColumnRenamed con la siguiente sintaxis
sales_df.withColumnRenamed('Region','Logist_Area').show(truncate=False)

root
 |-- Order_ID: integer (nullable = true)
 |-- Order_Date: date (nullable = true)
 |-- Item_Type: string (nullable = true)
 |-- Units_Sold: integer (nullable = true)
 |-- Unit_Price: double (nullable = true)
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Ship_Date: date (nullable = true)

None
+---------+----------+---------------+----------+----------+---------------------------------+----------------------+----------+
|Order_ID |Order_Date|Item_Type      |Units_Sold|Unit_Price|Logist_Area                      |Country               |Ship_Date |
+---------+----------+---------------+----------+----------+---------------------------------+----------------------+----------+
|535113847|2014-10-08|Snacks         |934       |152.58    |Middle East and North Africa     |Azerbaijan            |2014-10-23|
|874708545|2015-02-22|Cosmetics      |4551      |437.2     |Central America and the Caribbean|Panama                |2015-02-27|
|854349935|2015-12-09

In [0]:
# creamos un resumen del dataframe para no perder el original
resumen_df = sales_df.withColumn("Total_Price", expr("Units_Sold *  Unit_Price"))
# también podemos eliminar columnas, tanto las determinadas como las que hemos creado
resumen_df2 = resumen_df.drop('Unit_Price','Region')
resumen_df2.printSchema()
resumen_df2.show(10, truncate=False)

root
 |-- Order_ID: integer (nullable = true)
 |-- Order_Date: date (nullable = true)
 |-- Item_Type: string (nullable = true)
 |-- Units_Sold: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Ship_Date: date (nullable = true)
 |-- Total_Price: double (nullable = true)

+---------+----------+-------------+----------+---------------------+----------+------------------+
|Order_ID |Order_Date|Item_Type    |Units_Sold|Country              |Ship_Date |Total_Price       |
+---------+----------+-------------+----------+---------------------+----------+------------------+
|535113847|2014-10-08|Snacks       |934       |Azerbaijan           |2014-10-23|142509.72         |
|874708545|2015-02-22|Cosmetics    |4551      |Panama               |2015-02-27|1989697.2         |
|854349935|2015-12-09|Fruits       |9986      |Sao Tome and Principe|2016-01-18|93169.38          |
|892836844|2014-09-17|Personal Care|9118      |Sao Tome and Principe|2014-10-12|745214.14         |
|1292806

In [0]:
df.show()
# también podemos eliminar aquellos valores nulos, identificando las columnas que queremos hacer el drop
not_null_df = df.dropna(subset='salary')
not_null_df.show()

+---------+-------------+-----+------+------+
|firstname|     lastname|   id|gender|salary|
+---------+-------------+-----+------+------+
|     null|     Smith   |36636|     M|  3500|
|  Michael|         Rose|40288|     M|  4750|
|   Robert|     Williams|42114|     M|  null|
|    Maria|    Jones    |39192|     F|  4000|
+---------+-------------+-----+------+------+

+---------+-------------+-----+------+------+
|firstname|     lastname|   id|gender|salary|
+---------+-------------+-----+------+------+
|     null|     Smith   |36636|     M|  3500|
|  Michael|         Rose|40288|     M|  4750|
|    Maria|    Jones    |39192|     F|  4000|
+---------+-------------+-----+------+------+



In [0]:
# podemos cambiar el tipo de una columna en concreto con .cast() y el tipo entre '' para que se guarde. esto sirve si se ha hecho mal el inferScheme, para poder cambiar el tipo de datos de manera sencilla
sales_df.withColumn('Order_ID', col('Order_ID').cast('string')).printSchema()


root
 |-- Order_ID: string (nullable = true)
 |-- Order_Date: date (nullable = true)
 |-- Item_Type: string (nullable = true)
 |-- Units_Sold: integer (nullable = true)
 |-- Unit_Price: double (nullable = true)
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Ship_Date: date (nullable = true)



In [0]:
# para hacerlo bonito, podemos eliminar espacio de las columnas de texto con .ltrim, .rtrim y .trim
from pyspark.sql.functions import ltrim, rtrim, trim
# mostramos el dataframe para saber dónde tenemos espacios de más
df.show()
# corregimos con trim en la columna
corregido2 = df.withColumn('lastname', trim(col('lastname')))
# mostrmos el dataframe corregido
corregido2.show()


+---------+-------------+-----+------+------+
|firstname|     lastname|   id|gender|salary|
+---------+-------------+-----+------+------+
|     null|     Smith   |36636|     M|  3500|
|  Michael|         Rose|40288|     M|  4750|
|   Robert|     Williams|42114|     M|  null|
|    Maria|    Jones    |39192|     F|  4000|
+---------+-------------+-----+------+------+

+---------+--------+-----+------+------+
|firstname|lastname|   id|gender|salary|
+---------+--------+-----+------+------+
|     null|   Smith|36636|     M|  3500|
|  Michael|    Rose|40288|     M|  4750|
|   Robert|Williams|42114|     M|  null|
|    Maria|   Jones|39192|     F|  4000|
+---------+--------+-----+------+------+



##EXTRA: ESCRIBIR (GUARDAR) DATAFRAME RESULTADO, FORMATO PARQUET (DISTRIBUIDO)

In [0]:
# podemos guardar el datagrame del resultado en formato parquet en nuestro path de databricks
path = 'dbfs:/FileStore/shared_uploads/jesuspt98@gmail.com/sales_clean'
(sales_df.write
       	.format("parquet")
            	.mode("overwrite")
            	.option("compression", "snappy")
            	.save(path))


In [0]:
%fs ls 'dbfs:/FileStore/shared_uploads/jesuspt98@gmail.com/sales_clean'


path,name,size,modificationTime
dbfs:/FileStore/shared_uploads/jesuspt98@gmail.com/sales_clean/_SUCCESS,_SUCCESS,0,1697791607000
dbfs:/FileStore/shared_uploads/jesuspt98@gmail.com/sales_clean/_committed_7240800930604064197,_committed_7240800930604064197,224,1697791607000
dbfs:/FileStore/shared_uploads/jesuspt98@gmail.com/sales_clean/_started_7240800930604064197,_started_7240800930604064197,0,1697791606000
dbfs:/FileStore/shared_uploads/jesuspt98@gmail.com/sales_clean/part-00000-tid-7240800930604064197-d39c34be-b41b-47eb-b389-1faae9ffec3e-162-1-c000.snappy.parquet,part-00000-tid-7240800930604064197-d39c34be-b41b-47eb-b389-1faae9ffec3e-162-1-c000.snappy.parquet,662217,1697791607000
dbfs:/FileStore/shared_uploads/jesuspt98@gmail.com/sales_clean/part-00001-tid-7240800930604064197-d39c34be-b41b-47eb-b389-1faae9ffec3e-163-1-c000.snappy.parquet,part-00001-tid-7240800930604064197-d39c34be-b41b-47eb-b389-1faae9ffec3e-163-1-c000.snappy.parquet,588435,1697791607000
